In [1]:
!pip install beautifulsoup4
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
import os
from sklearn.cluster import KMeans
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

     |████████████████████████████████| 122kB 6.3MB/s eta 0:00:01


In [2]:
#uploaded indianapolis zip codes
zipcodes_df = pd.read_csv('indy_zip_codes.csv')

In [3]:
zipcodes_df.head()

,Zip,City,State,Area Code,County,Latitude,Longitude
0,46201,Indianapolis,IN,317,Marion,39.775,-86.110
1,46202,Indianapolis,IN,317,Marion,39.790,-86.160
2,46203,Indianapolis,IN,317,Marion,39.743,-86.104
3,46204,Indianapolis,IN,317,Marion,39.775,-86.159
4,46205,Indianapolis,IN,317,Marion,39.828,-86.133


In [4]:
CLIENT_ID = 'BCDLA13GUNOML3D4UFMSMKC3ECUGTCOMVXB5C1ZKLTULZZHJ' # your Foursquare ID
CLIENT_SECRET = '0HXQSO4JFJD0VLSDVD55DU3XKLLWSDXZYXR5CUUS5ETS4GKV' # your Foursquare Secret
VERSION = '20180604'

In [5]:
#venue category code: grocery store, farmer's market, health food store, organic grocery, supermarket
categoryId = '4bf58dd8d48988d118951735,4bf58dd8d48988d1fa941735,52f2ab2ebcbc57f1066b8b45,52f2ab2ebcbc57f1066b8b46'
#food deserts are described as living more than 1 mile from a grocery store; 1 mile = 1609.34 miles
radius=1609.34
LIMIT=100

# create the API request URL, TEST
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
CLIENT_ID, CLIENT_SECRET, 39.775, -86.110, VERSION, categoryId, radius, LIMIT)

#look at results
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)

# filter columns
filtered_columns = ['venue.name', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,lat,lng
0,Pogue's Run Grocer,39.781468,-86.115672
1,Save-A-Lot,39.767601,-86.102792
2,Kroger Grocery Delivery and Pickup,39.780108,-86.094324
3,Kroger,39.779492,-86.093207
4,2 Amigos Mexican Grocery,39.760695,-86.112675


In [6]:
#food deserts are described as living more than 1 mile from a grocery store
def getNearbyGroceries(zipcode, latitudes, longitudes):
    #venue category code: grocery store, farmer's market, health food store, organic grocery, supermarket
    categoryId = '4bf58dd8d48988d118951735,4bf58dd8d48988d1fa941735,52f2ab2ebcbc57f1066b8b45,52f2ab2ebcbc57f1066b8b46'
    #food deserts are described as living more than 1 mile from a grocery store; 1 mile = 1609.34 miles
    radius=1609.34
    LIMIT=100
    
    grocery_list = []
    for zipcode, lat, lng in zip(zipcode, latitudes, longitudes):
        print(zipcode)
            
        # create the API request URL, TEST
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, categoryId, radius, LIMIT)
            
        #look at results
        venues = requests.get(url).json()['response']['groups'][0]['items']

        # return information about each venue
        grocery_list.append([(
            zipcode,
            lat,
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in venues])

    nearby_groceries = pd.DataFrame([item for grocery_list in grocery_list for item in grocery_list])
    nearby_groceries.columns = ['Zipcode',
                  'Zipcode Latitude', 
                  'Zipcode Longitude',
                  'Grocery Store', 
                  'Grocery Store Latitude', 
                  'Grocery Store Longitude',
                  'Category']
    
    return(nearby_groceries)

In [7]:
groceries = getNearbyGroceries(zipcode = zipcodes_df['Zip'],
                                   latitudes = zipcodes_df['Latitude'],
                                   longitudes = zipcodes_df['Longitude']
                                  )

46201
46202
46203
46204
46205
46206
46207
46208
46209
46211
46214
46216
46217
46218
46219
46220
46221
46222
46223
46224
46225
46226
46227
46228
46229
46230
46231
46234
46235
46236
46237
46239
46240
46241
46242
46244
46247
46249
46250
46251
46253
46254
46255
46256
46259
46260
46266
46268
46274
46275
46277
46278
46280
46282
46283
46285
46290
46291
46295
46296
46298


In [8]:
groceries.head()

,Zipcode,Zipcode Latitude,Zipcode Longitude,Grocery Store,Grocery Store Latitude,Grocery Store Longitude,Category
0,46201,39.775,-86.11,Pogue's Run Grocer,39.781468,-86.115672,Grocery Store
1,46201,39.775,-86.11,Save-A-Lot,39.767601,-86.102792,Grocery Store
2,46201,39.775,-86.11,Kroger Grocery Delivery and Pickup,39.780108,-86.094324,Grocery Store
3,46201,39.775,-86.11,Kroger,39.779492,-86.093207,Grocery Store
4,46201,39.775,-86.11,2 Amigos Mexican Grocery,39.760695,-86.112675,Grocery Store


In [36]:
groceries_grouped = groceries.groupby('Zipcode').count().reset_index()
groceries_grouped['Zipcode'] = groceries_grouped['Zipcode'].astype(str)
groceries_grouped.head()

,Zipcode,Zipcode Latitude,Zipcode Longitude,Grocery Store,Grocery Store Latitude,Grocery Store Longitude,Category
0,46201,5,5,5,5,5,5
1,46202,2,2,2,2,2,2
2,46204,7,7,7,7,7,7
3,46205,2,2,2,2,2,2
4,46206,2,2,2,2,2,2


In [16]:
!wget --quiet https://opendata.arcgis.com/datasets/bba987ced0cf4b7086650e3656b30d69_7.geojson
print('GeoJSON file downloaded!')

GeoJSON file downloaded!


In [28]:
indiana_geo = r'https://opendata.arcgis.com/datasets/bba987ced0cf4b7086650e3656b30d69_7.geojson' # geojson file

In [44]:
#create folium map of indianapolis
indy_map = folium.Map(location = [39.775, -86.110], zoom_start = 12)

#create choropleth map based off grocery store count
indy_map.choropleth(
    geo_data=indiana_geo,
    data = groceries_grouped,
    columns = ['Zipcode','Category'],
    key_on = 'feature.properties.ZIPCODE',
    fill_color = 'YlOrRd', 
    fill_opacity = 0.7, 
    line_opacity = 0.2,
    legend_name='Grocery Stores in Indianapolis Zip Codes'
)

indy_map